<a href="https://colab.research.google.com/github/aparajitakar/nfl_yards_gained/blob/main/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# Script directory
main_dir = '/content/gdrive/Shareddrives/CSCI 5523 Project/dev/csv/'

Mounted at /content/gdrive


In [ ]:
import pandas as pd
#from sklearn.ensemble import RandomForestClassifier
df_train = pd.read_csv(main_dir + "merged/merged_train.csv").dropna()
df_test = pd.read_csv(main_dir + "merged/merged_test.csv").dropna()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280336 entries, 0 to 334288
Data columns (total 53 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   unique_idx              280336 non-null  int64  
 1   GameId                  280336 non-null  int64  
 2   PlayId                  280336 non-null  int64  
 3   A                       280336 non-null  float64
 4   DisplayName             280336 non-null  object 
 5   GameClock               280336 non-null  float64
 6   HomeScoreBeforePlay     280336 non-null  int64  
 7   DefendersInTheBox       280336 non-null  float64
 8   Position                280336 non-null  object 
 9   YardBins                280336 non-null  object 
 10  HumidityNoNull          280336 non-null  float64
 11  S                       280336 non-null  float64
 12  S_binned                280336 non-null  int64  
 13  Quarter                 280336 non-null  int64  
 14  OP_RB               

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49676 entries, 0 to 60103
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   unique_idx              49676 non-null  int64  
 1   GameId                  49676 non-null  int64  
 2   PlayId                  49676 non-null  int64  
 3   A                       49676 non-null  float64
 4   DisplayName             49676 non-null  object 
 5   GameClock               49676 non-null  float64
 6   HomeScoreBeforePlay     49676 non-null  int64  
 7   DefendersInTheBox       49676 non-null  float64
 8   Position                49676 non-null  object 
 9   HumidityNoNull          49676 non-null  float64
 10  YardBins                49676 non-null  object 
 11  S_binned                49676 non-null  int64  
 12  Quarter                 49676 non-null  int64  
 13  OP_RB                   49676 non-null  int64  
 14  OP_OL                   49676 non-null

In [ ]:
# Train
X_train = df_train.drop(["YardBins","DisplayName","unique_idx","DisplayName","GameClock","Unnamed: 0_x","Dis","Unnamed: 0_y"], axis = 1)
X_train = pd.get_dummies(X_train)
y_train = df_train[["YardBins"]]

# Test
X_test = df_test.drop(["YardBins","DisplayName","unique_idx","DisplayName","GameClock","Unnamed: 0_x","Dis","Unnamed: 0_y"], axis = 1)
X_test = pd.get_dummies(X_test)
y_test = df_test[["YardBins"]]

In [ ]:
# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( X_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_test.columns]

In [ ]:
#Decision Tree 
max_depth = 100
model = tree.DecisionTreeClassifier(random_state = 1, max_depth = max_depth)
#model = RandomForestClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [ ]:
print(accuracy_score(y_test, y_pred))

0.16688139141637812


In [ ]:
pred_train = model.predict(X_train)
print(accuracy_score(y_train, pred_train))

1.0


In [ ]:
df_pred = list(y_pred.copy())
dict_changename = {'(-inf, -1.0]': 1, '(-1.0, 0.0]': 2, '(0.0, 2.0]': 3, '(2.0, 4.0]': 4, '(4.0, 6.0]': 5, 
                      '(6.0, 8.0]': 6, '(8.0, 10.0]': 7,'(10.0, 15.0]': 8, '(15.0, 20.0]': 9, '(20.0, inf]': 10}
df_pred_new = [dict_changename.get(n, n) for n in df_pred]

In [ ]:
df_true =  list(y_test["YardBins"].copy())
df_true_new = [dict_changename.get(n, n) for n in df_true]

In [ ]:
import numpy as np
diff = abs(np.array(df_true_new) - np.array(df_pred_new))
df_describe = pd.DataFrame(diff)
df_describe[df_describe[0] <= 1].shape[0] / df_describe.shape[0]

0.5651018600531443

In [ ]:
df_describe[df_describe[0] <= 2].shape[0] / df_describe.shape[0]

0.8113374667847653

In [ ]:
target_names = ["Class 1", "Class 2","Class 3", "Class 4", "Class 5", "Class 6", "Class 7","Class 8","Class 9", "Class 10"]
print(classification_report(y_test, y_pred, target_names=target_names))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     Class 1       0.28      0.02      0.04      4290
     Class 2       0.11      0.00      0.01      5566
     Class 3       0.29      0.23      0.26     12320
     Class 4       0.10      0.01      0.02      2354
     Class 5       0.00      0.00      0.00       792
     Class 6       0.00      0.00      0.00     11462
     Class 7       0.00      0.00      0.00       990
     Class 8       0.14      0.79      0.24      6666
     Class 9       0.00      0.00      0.00      3410
    Class 10       0.00      0.00      0.00      1826

    accuracy                           0.17     49676
   macro avg       0.09      0.11      0.06     49676
weighted avg       0.13      0.17      0.10     49676



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Logistic Regression 

logit = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logit.fit(X_train,y_train)
y_pred = logit.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [ ]:
print(accuracy_score(y_test, y_pred))

0.24800708591674048


In [ ]:
pred_train = logit.predict(X_train)
print(accuracy_score(y_train,pred_train))

0.2462901660864106


In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV 
import numpy as np


In [ ]:
def clf_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

In [ ]:
lr = LogisticRegression(multi_class='multinomial')
param_grid = {'max_iter' : [2000],
              'penalty' : ['l1', 'l2'],
              'C' : np.logspace(-4, 4, 20),
              'solver' : ['lbfgs']}

clf_lr = GridSearchCV(lr, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_lr = clf_lr.fit(X_train,y_train)
clf_performance(best_clf_lr,'Logistic Regression')


Fitting 5 folds for each of 40 candidates, totalling 200 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-p

Logistic Regression
Best Score: 0.24629016614984384
Best Parameters: {'C': 0.0001, 'max_iter': 2000, 'penalty': 'l2', 'solver': 'lbfgs'}


In [ ]:
#Naive Bayes 

Nb = GaussianNB()
Nb.fit(X_train,y_train)
pred = Nb.predict(X_test)
print(accuracy_score(y_test, pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


0.24800708591674048


In [ ]:
#svc = SVC()
#svc.fit(X_train,y_train)
#y_pred = svc.predict(X_test)
#print(accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import lightgbm as lgb

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_y = le.fit_transform(y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
test_y = le.fit_transform(y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
gbm = lgb.LGBMRegressor()
gbm.fit(X_train,train_y)
y_pred = gbm.predict(X_test)


In [ ]:
y_pred

array([4.41203498, 4.41203498, 4.41203498, ..., 4.05707743, 3.9232227 ,
       4.1449148 ])

In [ ]:
df_train.head()

,unique_idx,GameId,PlayId,A,DisplayName,GameClock,HomeScoreBeforePlay,DefendersInTheBox,Position,YardBins,HumidityNoNull,S_binned,Quarter,OP_RB,OP_OL,OP_QB,OP_TE,OP_WR,OP_DL,OP_LB,OP_DB,Unnamed: 0_x,Team,Dis,PossessionTeam,VisitorScoreBeforePlay,DP_DL,DP_LB,DP_DB,DP_RB,DP_OL,Height_in_cms,WindSpeed_dc,StadiumType,HomeTeamAbbr,Unnamed: 0_y,X,Orientation,Season,Down,PlayDirection,PlayerWeight,VisitorTeamAbbr,Turf,WindDirection,IsRusher,Y,Dir,GameWeather,YardLine,Distance,OffenseFormation
0,1,2017090700,20170907000118,1.13,Eric Berry,854.0,0,6.0,SS,"(6.0, 8.0]",77.0,1,1,1,0,0,1,3,0,0,0,0,away,0.40,NE,0,2,3,6,0,0,182.88,8.0,outdoor,NE,0,73.91,81.99,2017,3,left,212,KC,Artificial,SW,False,32.64,198.70,clear,35,2,SHOTGUN
1,2,2017090700,20170907000118,1.35,Allen Bailey,854.0,0,6.0,DE,"(6.0, 8.0]",77.0,0,1,1,0,0,1,3,0,0,0,1,away,0.01,NE,0,2,3,6,0,0,190.50,8.0,outdoor,NE,1,74.67,27.61,2017,3,left,288,KC,Artificial,SW,False,33.20,202.73,clear,35,2,SHOTGUN
2,3,2017090700,20170907000118,0.59,Justin Houston,854.0,0,6.0,DE,"(6.0, 8.0]",77.0,1,1,1,0,0,1,3,0,0,0,2,away,0.31,NE,0,2,3,6,0,0,190.50,8.0,outdoor,NE,2,74.00,3.01,2017,3,left,270,KC,Artificial,SW,False,27.70,105.64,clear,35,2,SHOTGUN
3,4,2017090700,20170907000118,0.54,Derrick Johnson,854.0,0,6.0,ILB,"(6.0, 8.0]",77.0,0,1,1,0,0,1,3,0,0,0,3,away,0.02,NE,0,2,3,6,0,0,190.50,8.0,outdoor,NE,3,71.46,359.77,2017,3,left,245,KC,Artificial,SW,False,35.42,164.31,clear,35,2,SHOTGUN
4,5,2017090700,20170907000118,2.43,Ron Parker,854.0,0,6.0,FS,"(6.0, 8.0]",77.0,2,1,1,0,0,1,3,0,0,0,4,away,0.16,NE,0,2,3,6,0,0,182.88,8.0,outdoor,NE,4,69.32,12.63,2017,3,left,206,KC,Artificial,SW,False,24.00,95.01,clear,35,2,SHOTGUN


In [ ]:
X_train = df_train.drop(["DisplayName","unique_idx","GameClock","Unnamed: 0_x","Dis","Unnamed: 0_y","Season"], axis = 1)

In [ ]:
#X_train.head(22)

In [ ]:
print(f"Total games: {X_train.GameId.nunique()}")
print(f"Total plays: {X_train.PlayId.nunique()}") #PlayId is important

Total games: 401
Total plays: 12746


In [ ]:
print(f"Total Possession Teams: {X_train.PossessionTeam.nunique()}")

Total Possession Teams: 32


In [ ]:
X_train.groupby(["PlayId","PossessionTeam"]).GameId.count().value_counts()

22    12718
21       25
1         2
13        1
Name: GameId, dtype: int64

In [ ]:
cols = ["Dir","Y","IsRusher","PlayerWeight","Orientation","X","Height_in_cms","A","Position","S_binned"]
train_x = X_train.drop(cols,axis = 1).drop_duplicates(subset = "PlayId")
train_y = X_train[["YardBins","PlayId"]].drop_duplicates(subset = "PlayId")

In [ ]:
train_x.head()

,GameId,PlayId,HomeScoreBeforePlay,DefendersInTheBox,YardBins,HumidityNoNull,Quarter,OP_RB,OP_OL,OP_QB,OP_TE,OP_WR,OP_DL,OP_LB,OP_DB,Team,PossessionTeam,VisitorScoreBeforePlay,DP_DL,DP_LB,DP_DB,DP_RB,DP_OL,WindSpeed_dc,StadiumType,HomeTeamAbbr,Down,PlayDirection,VisitorTeamAbbr,Turf,WindDirection,GameWeather,YardLine,Distance,OffenseFormation
0,2017090700,20170907000118,0,6.0,"(6.0, 8.0]",77.0,1,1,0,0,1,3,0,0,0,away,NE,0,2,3,6,0,0,8.0,outdoor,NE,3,left,KC,Artificial,SW,clear,35,2,SHOTGUN
22,2017090700,20170907000395,7,7.0,"(6.0, 8.0]",77.0,1,1,0,0,3,1,0,0,0,away,NE,0,3,4,4,0,0,8.0,outdoor,NE,1,right,KC,Artificial,SW,clear,10,1,SINGLEBACK
44,2017090700,20170907000473,7,5.0,"(8.0, 10.0]",77.0,1,1,0,0,1,3,0,0,0,away,KC,0,3,2,6,0,0,8.0,outdoor,NE,3,left,KC,Artificial,SW,clear,10,10,SINGLEBACK
66,2017090700,20170907000516,7,8.0,"(-inf, -1.0]",77.0,1,1,0,0,2,2,0,0,0,away,KC,0,3,2,6,0,0,8.0,outdoor,NE,2,left,KC,Artificial,SW,clear,9,9,SINGLEBACK
88,2017090700,20170907000653,7,9.0,"(-1.0, 0.0]",77.0,1,1,6,0,2,1,0,0,0,away,NE,7,2,3,6,0,0,8.0,outdoor,NE,4,left,KC,Artificial,SW,clear,25,10,SHOTGUN


In [ ]:
train_x.count()

GameId                    12746
PlayId                    12746
HomeScoreBeforePlay       12746
DefendersInTheBox         12746
YardBins                  12746
HumidityNoNull            12746
Quarter                   12746
OP_RB                     12746
OP_OL                     12746
OP_QB                     12746
OP_TE                     12746
OP_WR                     12746
OP_DL                     12746
OP_LB                     12746
OP_DB                     12746
Team                      12746
PossessionTeam            12746
VisitorScoreBeforePlay    12746
DP_DL                     12746
DP_LB                     12746
DP_DB                     12746
DP_RB                     12746
DP_OL                     12746
WindSpeed_dc              12746
StadiumType               12746
HomeTeamAbbr              12746
Down                      12746
PlayDirection             12746
VisitorTeamAbbr           12746
Turf                      12746
WindDirection             12746
GameWeat

In [ ]:
train_y.count()

YardBins    12746
PlayId      12746
dtype: int64

In [ ]:
# Train

X_train = df_train.drop(["DisplayName","unique_idx","GameClock","Unnamed: 0_x","Dis","Unnamed: 0_y","GameId"], axis = 1)
X_test = df_test.drop(["DisplayName","unique_idx","GameClock","Unnamed: 0_x","Dis","Unnamed: 0_y","GameId"], axis = 1)

cols = ["Dir","Y","IsRusher","PlayerWeight","Orientation","X","Height_in_cms","A","Position","S_binned"]
#cols = ["Dir","Y","IsRusher","PlayerWeight","Orientation","X","Height_in_cms","A","Position","S_binned","Yardbins","PlayId"]

train_x = X_train.drop(cols,axis = 1).drop_duplicates(subset = "PlayId")
#train_x = X_train[cols]
train_y = X_train[["YardBins","PlayId"]].drop_duplicates(subset = "PlayId")

train_x = train_x.drop(["YardBins","PlayId"],axis = 1)
train_x = pd.get_dummies(train_x)
train_y = train_y[["YardBins"]]

# Test

test_x = X_test.drop(cols,axis = 1).drop_duplicates(subset = "PlayId")
test_y = test_x[["YardBins","PlayId"]].drop_duplicates(subset = "PlayId")

test_x = test_x.drop(["YardBins","PlayId"],axis = 1)
test_x = pd.get_dummies(test_x)
test_y = test_y[["YardBins"]]

In [ ]:
# Get missing columns in the training test
missing_cols = set(train_x.columns ) - set(test_x.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    test_x[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
test_x = test_x[test_x.columns]

In [ ]:
RF = RandomForestClassifier()
RF.fit(train_x,train_y)
prediction = RF.predict(test_x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [ ]:
target_names = ["Class 1", "Class 2","Class 3", "Class 4", "Class 5", "Class 6", "Class 7","Class 8","Class 9", "Class 10"]
print(classification_report(test_y, prediction,target_names=target_names))
#

              precision    recall  f1-score   support

     Class 1       0.08      0.12      0.09       195
     Class 2       0.11      0.03      0.05       253
     Class 3       0.25      0.83      0.38       560
     Class 4       0.00      0.00      0.00       107
     Class 5       0.00      0.00      0.00        36
     Class 6       0.22      0.00      0.01       521
     Class 7       0.00      0.00      0.00        45
     Class 8       0.00      0.00      0.00       303
     Class 9       0.00      0.00      0.00       155
    Class 10       0.00      0.00      0.00        83

    accuracy                           0.22      2258
   macro avg       0.07      0.10      0.05      2258
weighted avg       0.13      0.22      0.11      2258



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
